Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [140]:
import random
from random import choices
from copy import copy
import numpy as np
import lab9_lib
from tqdm.notebook import tqdm

# Local Search:
Local search is a heuristic method for solving computationally hard optimization problems. Local search can be used on problems that can be formulated as finding a solution maximizing a criterion among a number of candidate solutions. Local search algorithms move from solution to solution in the space of candidate solutions (the search space) by applying local changes, until a solution deemed optimal is found or a time bound is elapsed.

In [141]:
l = 1000
problems = [1, 2, 5, 10]
half_pop_size = 5
µ = 2 * half_pop_size

## Implementation:
Idea: with the fitness function I can see which pieces of string are important and which are not, and therefore I can preserve the important pieces and throw away the useless ones.

IMPORTANT: PROMOTE DIVERSITY (I can do it in the selection, crossover and mutation)
* distance metric: 
  - how far the individual is from a subset of the population to the whole population
  - from a single individual
* property of the population

3 levels of diversity:
* phenotype
* genotype
* fitness

In [142]:
# invece che fare find distribution potrei fare direttamente una funzione che mi prende inf e sup di ogni serie consecutiva di 1
# oppure fare direttamente l'and bit a bit e contare il numero di 1
# L'and conta solo quali 1 sono in comune, ma non considera la similarità con gli 0
# quindi per vedere se due vettori sono simili devo fare uno xor, perché se i due elementi sono diversi, allora il risultato è 1, altrimenti 0
# poi conto gli 1 che indicano che due elem sono diversi

# Evaluate diversity (how many different elements we have in the genome)
def evaluate_diversity(e1, e2):
    xor_res = e1 ^ e2
    return np.sum(xor_res) / l


In [143]:
def init_population():
    return np.array([(np.array(choices([0, 1], k=l)), 0.0) for _ in range(µ)], dtype=object)

def evaluate_population(population, fitness):
    return np.array([(individual[0], fitness(individual[0])) for individual in population], dtype=object)

def select_with_replacement(population):
    # select a random individual from the population
    # find the individual with 
    # the highest fitness, 
    # the vector with the most distinct distribution of ones compared to the others
    div_matr = np.zeros((µ, µ))
    for i1, p1 in enumerate(population):
        for i2, p2 in enumerate(population):
            if i1 != i2:
                #the matrix is not symmetric since is added only for p2[1]
                div_matr[i1][i2] = (evaluate_diversity(p1[0], p2[0]) + p2[1]) / 2

    # find index of the individuals with highest fitness and highest diversity
    i1, i2 = np.unravel_index(np.argmax(div_matr), div_matr.shape)

    return population[i1], population[i2]

def crossover_cyclic_shift(parent1, parent2):
    # a two (rand) point crossover for now
    # swapping of two substrings of the same len but in random position
    # I want try to implement a circular translated swapping
    v = parent1[0]
    w = parent2[0]
    c = random.randint(0, l)
    d = random.randint(0, l)
    s = random.randint(0, l)
    if c < d:
        for i in range(c, d):
            v[i%l], w[(i+s)%l] = w[(i+s)%l], v[i%l]
    else:
        for i in range(c, d+l):
            v[i%l], w[(i+s)%l] = w[(i+s)%l], v[i%l]
    return (v, 0.0), (w, 0.0)

def std_crossover(parent1, parent2):
    # a two (rand) point crossover for now
    # swapping of two substrings of the same len but in random position
    # I want try to implement a circular translated swapping
    v = parent1[0]
    w = parent2[0]
    c = random.randint(0, l)
    d = random.randint(0, l)
    if c > d:
        c, d = d, c
    if c!=d:
        v[c:d], w[c:d] = w[c:d], v[c:d]
    return (v, 0.0), (w, 0.0)

def mutate(individual):
    # bit flip mutation for now
    p = 0.5
    v = individual[0]
    for i in range(l):
        if p >= random.random():
            v[i] = 1 - v[i]
    return individual

In [144]:
def genetic_algorithm(fitness, crossover):
    Best = None
    # 1. Initialize population
    population = init_population()
    population = evaluate_population(population, fitness)
    # 2. Repeat
    found = -1
    for i in tqdm(range(100)):
        for p in population:
            if Best is None or p[1] > Best[1]:
                Best = p
                found = fitness._calls
        
        if Best is not None and Best[1]==1:
            break
        
        q = list()
        for _ in range(µ//2):
            # 2.1 Select parents
            parent_a, parent_b = select_with_replacement(population)
            # 2.2 Crossover
            child_a, child_b = crossover(copy(parent_a), copy(parent_b))
            
            # 2.3 Mutate
            mutated_a = mutate(child_a)
            mutated_b = mutate(child_b)
            q.append(mutated_a)
            q.append(mutated_b)
            
        population = evaluate_population(q, fitness)
    
    # 4. Return best individual
    return Best, found

In [145]:
for _ in range(3):
    my_list = list()
    found = -1
    fitness = None
    for prob in problems:
        fitness = lab9_lib.make_problem(prob)
        b, found = genetic_algorithm(fitness, crossover_cyclic_shift)
        my_list.append((prob, b[1], fitness.calls))
    for m in my_list:
        print(f"Problem\t{m[0]}:\t{m[1]:.2%},\tCalls:\t{m[2]},\tBest found at \t{found} fitness calls")
    print("-------------------------------------")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Problem	1:	54.00%,	Calls:	1010,	Best found at 	440 fitness calls
Problem	2:	52.40%,	Calls:	1010,	Best found at 	440 fitness calls
Problem	5:	38.31%,	Calls:	1010,	Best found at 	440 fitness calls
Problem	10:	16.21%,	Calls:	1010,	Best found at 	440 fitness calls
-------------------------------------


...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Problem	1:	54.10%,	Calls:	1010,	Best found at 	490 fitness calls
Problem	2:	50.40%,	Calls:	1010,	Best found at 	490 fitness calls
Problem	5:	21.45%,	Calls:	1010,	Best found at 	490 fitness calls
Problem	10:	16.79%,	Calls:	1010,	Best found at 	490 fitness calls
-------------------------------------


In [146]:
for _ in range(3):
    my_list = list()
    found = -1
    fitness = None
    for prob in problems:
        fitness = lab9_lib.make_problem(prob)
        b, found = genetic_algorithm(fitness, std_crossover)
        my_list.append((prob, b[1], fitness.calls))
    for m in my_list:
        print(f"Problem\t{m[0]}:\t{m[1]:.2%},\tCalls:\t{m[2]},\tBest found at \t{found} fitness calls")
    print("-------------------------------------")

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Problem	1:	55.30%,	Calls:	1010,	Best found at 	600 fitness calls
Problem	2:	53.20%,	Calls:	1010,	Best found at 	600 fitness calls
Problem	5:	31.00%,	Calls:	1010,	Best found at 	600 fitness calls
Problem	10:	16.80%,	Calls:	1010,	Best found at 	600 fitness calls
-------------------------------------


...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Problem	1:	54.00%,	Calls:	1010,	Best found at 	150 fitness calls
Problem	2:	52.40%,	Calls:	1010,	Best found at 	150 fitness calls
Problem	5:	30.40%,	Calls:	1010,	Best found at 	150 fitness calls
Problem	10:	17.15%,	Calls:	1010,	Best found at 	150 fitness calls
-------------------------------------


In [147]:
print("P1")

fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{''.join(str(g) for g in ind[0:5])}...: {fitness(ind):.2%}")

print(fitness.calls)

print("-------------------------------------")
print("P2")
fitness = lab9_lib.make_problem(2)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{''.join(str(g) for g in ind[0:5])}...: {fitness(ind):.2%}")

print(fitness.calls)

print("-------------------------------------")
print("P5")
fitness = lab9_lib.make_problem(5)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{''.join(str(g) for g in ind[0:5])}...: {fitness(ind):.2%}")

print(fitness.calls)

print("-------------------------------------")
print("P10")
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=1000)
    print(f"{''.join(str(g) for g in ind[0:5])}...: {fitness(ind):.2%}")

print(fitness.calls)

P1
01001...: 49.10%
11011...: 49.50%
10001...: 49.10%
00110...: 52.40%
01101...: 50.50%
10110...: 47.10%
10110...: 51.50%
11111...: 49.90%
01100...: 52.50%
01101...: 54.50%
10
-------------------------------------
P2
00000...: 23.24%
10001...: 50.60%
00001...: 23.85%
10111...: 23.52%
00110...: 23.72%
10011...: 23.46%
11000...: 22.61%
01100...: 24.71%
10010...: 23.75%
00000...: 22.90%
10
-------------------------------------
P5
01110...: 9.96%
10110...: 10.35%
10001...: 10.43%
10011...: 9.54%
00110...: 21.45%
10001...: 10.02%
01010...: 9.38%
11100...: 10.13%
00000...: 9.13%
10110...: 9.74%
10
-------------------------------------
P10
10010...: 5.21%
00101...: 5.31%
11111...: 10.79%
01000...: 10.41%
10001...: 4.90%
01101...: 5.47%
01011...: 5.26%
01111...: 4.90%
01110...: 5.44%
00110...: 5.28%
10


In [148]:
fitness = lab9_lib.make_problem(1)
for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

11110111110110110100011110101110000110011011000110: 60.00%
00101011010110001011110111011001001010001001101111: 54.00%
10100100101000010000011110001011100110011111101011: 50.00%
01111010011101111010100101100010111011000010000100: 50.00%
00001110111010110010101010100001010000000001100101: 40.00%
10100011001110101110010101110001110111000110110011: 56.00%
01111110001100000100011000000111000100001110011011: 44.00%
10111001011011011100011011111110001100001011000100: 54.00%
01010110011101010000000011110110111110101010100110: 52.00%
01011110011011100110101010100000110111111101011100: 58.00%
10
